#### Calculate total GW and km needed for grid in NS by cases

In [23]:
import os
import sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import pandas as pd
from constants import OFFSHORE_AREAS

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
DF_BY_CASES = dict({})

Cable length

In [24]:
cable_length = pd.read_csv("../../EMPIRE_extension/Output/directional_lines.csv")
cable_length["FromNode"] = cable_length["FromNode"].apply(lambda n: n.replace(" ", ""))
cable_length["ToNode"] = cable_length["ToNode"].apply(lambda n: n.replace(" ", ""))
cable_length

,FromNode,ToNode,lineLength in km,LineType
0,MorayFirth,FirthofForth,215.667240,HVDC_Cable
1,MorayFirth,DoggerBank,509.370695,HVDC_Cable
2,FirthofForth,DoggerBank,333.632410,HVDC_Cable
3,MorayFirth,Hornsea,586.957998,HVDC_Cable
4,FirthofForth,Hornsea,387.595780,HVDC_Cable
...,...,...,...,...
388,EnergyhubCentral,GreatBrit.,491.825549,HVDC_OffshoreToOnshore
389,EnergyhubCentral,Denmark,376.970459,HVDC_OffshoreToOnshore
390,EnergyhubCentral,Netherlands,470.165660,HVDC_OffshoreToOnshore
391,EnergyhubCentral,Belgium,614.084832,HVDC_OffshoreToOnshore


In [25]:
for case in CASES:
    _df = pd.read_csv(f'../Results/{case}/results_output_transmission.csv')
    _df = _df[(_df["BetweenNode"].isin(OFFSHORE_AREAS)) | (_df["AndNode"].isin(OFFSHORE_AREAS))]
    _df = _df.rename(columns={"BetweenNode": "FromNode", "AndNode": "ToNode"})
    _df = _df.merge(cable_length, on=["FromNode", "ToNode"], how="left")
    _df["Transmission MW km"] = _df["transmissionInstalledCap_MW"] * _df["lineLength in km"]
    DF_BY_CASES[case] = _df

DF_BY_CASES

{'Base':               FromNode        ToNode     Period  transmissionInvCap_MW  \
 0           MorayFirth  FirthofForth  2020-2025           4.018470e-04   
 1           MorayFirth  FirthofForth  2025-2030           5.106020e-05   
 2           MorayFirth  FirthofForth  2030-2035           1.122781e-03   
 3           MorayFirth  FirthofForth  2035-2040           1.748220e+01   
 4           MorayFirth  FirthofForth  2040-2045           1.894376e+02   
 ...                ...           ...        ...                    ...   
 3139  EnergyhubCentral       Germany  2035-2040           1.613436e+03   
 3140  EnergyhubCentral       Germany  2040-2045           6.646874e+03   
 3141  EnergyhubCentral       Germany  2045-2050           7.535213e-04   
 3142  EnergyhubCentral       Germany  2050-2055           3.552977e-07   
 3143  EnergyhubCentral       Germany  2055-2060           2.322881e-05   
 
       transmissionInstalledCap_MW  DiscountedInvestmentCost_EuroPerMW  \
 0              

Filter by case and period

In [54]:
FILTER_CASE = "NOgrid"
FILTER_PERIOD = "2045-2050"
MAX_CABLE_CAP = 2 # 1 or 2 GW

def calculate_cable_length_needed(df, cable_cap, period):
    total_length = 0
    for _, df_row in df.iterrows():
        num_cables = (df_row["transmissionInstalledCap_MW"] / 1000) // cable_cap + 1
        total_length += num_cables * df_row["lineLength in km"]
    start_year = 2023
    end_year = int(period.split("-")[1])
    num_years = end_year-start_year
    print(f"Total cable length needed until {end_year}: {round(total_length)} km with max {cable_cap}GW per cable")
    print(f"Average cable km installed per year until {end_year}: {round(total_length / num_years, 1)} km/yr")

filtered_df = DF_BY_CASES[FILTER_CASE]

# Threshold value 1 MW
filtered_df = filtered_df[(filtered_df["Period"] == FILTER_PERIOD) & (filtered_df["transmissionInstalledCap_MW"] > 1)]\
                            .sort_values("transmissionInstalledCap_MW", ascending=False)
print(f"Total cable capacitiy: {round(filtered_df["transmissionInstalledCap_MW"].sum() / 1000)} GW")
print(f"Total transmission in GW km: {round(filtered_df["Transmission MW km"].sum() / 1000)} GW km")
print(f"Average cable capacity between areas: {round(filtered_df["transmissionInstalledCap_MW"].mean() / 1000, 1)} GW\n")
calculate_cable_length_needed(filtered_df, MAX_CABLE_CAP, FILTER_PERIOD)
filtered_df.head(30)

Total cable capacitiy: 304 GW
Total transmission in GW km: 129735 GW km
Average cable capacity between areas: 4.1 GW

Total cable length needed until 2050: 79322 km with max 2GW per cable
Average cable km installed per year until 2050: 2937.9 km/yr


,FromNode,ToNode,Period,transmissionInvCap_MW,transmissionInstalledCap_MW,DiscountedInvestmentCost_EuroPerMW,transmissionExpectedAnnualVolume_GWh,ExpectedAnnualLosses_GWh,lineLength in km,LineType,Transmission MW km
3045,EnergyhubEU,Germany,2045-2050,1.094312e-04,29999.999967,8.467108e+00,192220.709390,5766.621282,565.794411,HVDC_OffshoreToOnshore,1.697383e+07
3141,EnergyhubCentral,Germany,2045-2050,1.279075e+03,29999.999953,1.312951e+08,180906.112532,5427.183376,750.895383,HVDC_OffshoreToOnshore,2.252686e+07
3109,EnergyhubCentral,GreatBrit.,2045-2050,2.312576e-06,19018.038881,1.555755e-01,95626.311245,2868.789337,491.825549,HVDC_OffshoreToOnshore,9.353557e+06
3093,EnergyhubNorth,Germany,2045-2050,7.642479e+03,16611.070854,1.074856e+09,115759.835711,3472.795071,1029.146418,HVDC_OffshoreToOnshore,1.709522e+07
3021,EnergyhubEU,Denmark,2045-2050,5.195659e-06,15828.021824,1.849374e-01,56778.722864,1703.361686,259.821659,HVDC_OffshoreToOnshore,4.112463e+06
3013,EnergyhubEU,GreatBrit.,2045-2050,1.422859e-06,13493.687487,1.055664e-01,67125.295842,2013.758875,542.500872,HVDC_OffshoreToOnshore,7.320337e+06
3117,EnergyhubCentral,Denmark,2045-2050,2.612542e+02,8021.295081,1.347829e+07,39781.357268,1193.440718,376.970459,HVDC_OffshoreToOnshore,3.023791e+06
3037,EnergyhubEU,Belgium,2045-2050,1.559430e+03,6769.861565,1.068715e+08,32747.094089,982.412823,501.044249,HVDC_OffshoreToOnshore,3.392000e+06
2829,HelgoländerBucht,Germany,2045-2050,0.000000e+00,6566.000000,0.000000e+00,35553.022075,1066.590662,439.059020,HVDC_OffshoreToOnshore,2.882862e+06
2621,EnergyhubEU,EnergyhubCentral,2045-2050,1.592079e-06,4999.999997,8.834127e-02,42512.195113,1275.365853,189.910300,HVDC_Cable,9.495515e+05
